# We will Perform item-item similarity using cosine similarity

In [91]:
# First let us import all required packages
import pandas as pd
import pandas as pd
from scipy.spatial.distance import cosine
#using encoding parameter as there are unicodes present in dataset
data=pd.read_csv('Online Retail.csv',encoding='unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [92]:
#Droping each column as only item,CustomerId and Quantity is needed
df=data.drop(['InvoiceNo','StockCode','InvoiceDate','UnitPrice','Country'],axis=1)
df.columns

Index(['Description', 'Quantity', 'CustomerID'], dtype='object')

In [93]:
df.rename(columns={'Description':'item','CustomerID':'Person'}, inplace=True)

col=list(df)
# col[0],col[1]=col[1],col[0]
# df.columns=col
df=df[['Person','item','Quantity']]
df.head()
#Taking only 300 items because as item increases time taken by algorithm increase
df=df.head(300)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
Person      300 non-null float64
item        300 non-null object
Quantity    300 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 7.1+ KB


In [94]:
df.item.value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     6
KNITTED UNION FLAG HOT WATER BOTTLE    5
SET 7 BABUSHKA NESTING BOXES           4
CREAM CUPID HEARTS COAT HANGER         4
HAND WARMER UNION JACK                 4
RED WOOLLY HOTTIE WHITE HEART.         4
GLASS STAR FROSTED T-LIGHT HOLDER      4
WHITE METAL LANTERN                    4
ALARM CLOCK BAKELIKE GREEN             4
ASSORTED COLOUR BIRD ORNAMENT          3
WOOD 2 DRAWER CABINET WHITE FINISH     3
WOODEN FRAME ANTIQUE WHITE             3
VINTAGE BILLBOARD LOVE/HATE MUG        3
VINTAGE BILLBOARD DRINK ME MUG         3
RED TOADSTOOL LED NIGHT LIGHT          3
RETRO COFFEE MUGS ASSORTED             3
ALARM CLOCK BAKELIKE RED               3
JAM MAKING SET WITH JARS               3
EDWARDIAN PARASOL RED                  3
SAVE THE PLANET MUG                    3
WOOD S/3 CABINET ANT WHITE FINISH      3
JAM MAKING SET PRINTED                 3
HAND WARMER RED POLKA DOT              3
SET OF 3 COLOURED  FLYING DUCKS        3
WOODEN PICTURE F

In [ ]:
# Now we will make all required functions to calculate similarity 

In [97]:

def createDf(dindex, dcolumns): # For creating pandas dataframe
    return pd.DataFrame(index=dindex,
                        columns=dcolumns)


#It takes a lot of time as it has nested loops.So we need to cut down the number of items.we have cut down into 210 items

# Similarity Measure using cosine distances
def findSimilarity(dataToBeFilled, dataUsed):       
    for i in range(0,len(dataToBeFilled.columns)):
        for j in range(0,len(dataToBeFilled.columns)): #Loop through the columns for each column
            dataToBeFilled.iloc[i,j] = cosine(dataUsed.iloc[:,i],dataUsed.iloc[:,j])
            print(i)
#     print('sumeet')
    return dataToBeFilled

# This function returns the neighbours.
def findNeighbours(neighbours, data, number):
    for i in range(0,len(data.columns)):
        neighbours.iloc[i,:number] = data.iloc[0:,i].sort_values(ascending=True)[1:number+1].index
#         print(i)
    return neighbours

#This method calls all other functions and return the data_neighbours.

# Item-based Collaborative Filtering
def itemBased(number, dataWide):    
    data_ib = dataWide.copy()
    data_ib = data_ib.reset_index() 
    data_ib = data_ib.drop("Person", axis=1)# droping user column
#     print(data_ib.head())
## Create a placeholder dataframe listing item vs. item with same columns name
    data_ibs = createDf(data_ib.columns,data_ib.columns)
#     print(data_ibs.columns)
#     print(data_ibs.head(10))
    data_ibs = findSimilarity(data_ibs,data_ib)
# number=number of neighbours we want   
    data_neighbours = createDf(data_ibs.columns,range(1,number+1))
    data_neighbours= findNeighbours(data_neighbours, data_ibs, number)
    return data_neighbours


#Now calling fuctions and passing dataset:

data=df
#Removing dublicated that is same_user and same_product
data= data.drop_duplicates(["Person", "item"])

data=data.dropna()

#Converting into pivot table
dataWide=data.pivot("Person", "item", "Quantity") 

dataWide.fillna(0, inplace=True) #Replace NA with 0

# 5 here denotes number of neighbours
#Calling itemBased function which returns neighbours
neighbours=itemBased(5, dataWide)



item                              SET 2 TEA TOWELS I LOVE LONDON   \
item                                                                
 SET 2 TEA TOWELS I LOVE LONDON                               NaN   
3 STRIPEY MICE FELTCRAFT                                      NaN   
3 TIER CAKE TIN GREEN AND CREAM                               NaN   
3 TIER CAKE TIN RED AND CREAM                                 NaN   
5 HOOK HANGER MAGIC TOADSTOOL                                 NaN   
60 TEATIME FAIRY CAKE CASES                                   NaN   
AIRLINE LOUNGE,METAL SIGN                                     NaN   
ALARM CLOCK BAKELIKE GREEN                                    NaN   
ALARM CLOCK BAKELIKE IVORY                                    NaN   
ALARM CLOCK BAKELIKE ORANGE                                   NaN   

item                             3 STRIPEY MICE FELTCRAFT  \
item                                                        
 SET 2 TEA TOWELS I LOVE LONDON                  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
1

29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
3

42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
4

...

164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
164
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165
165


173
173
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
174
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175
175


183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184


193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
193
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194


203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
203
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204


In [98]:
#The neighbours calculated are
neighbours.head(10)

,1,2,3,4,5
item,,,,,
SET 2 TEA TOWELS I LOVE LONDON,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW SPACEBOY,MINI PAINT SET VINTAGE,PANDA AND BUNNIES STICKER SHEET,POSTAGE
3 STRIPEY MICE FELTCRAFT,3 STRIPEY MICE FELTCRAFT,STAR DECORATION PAINTED ZINC,PICNIC BASKET WICKER SMALL,TRADITIONAL WOODEN SKIPPING ROPE,PARTY CONES CARNIVAL ASSORTED
3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,TOY TIDY PINK POLKADOT,RECYCLING BAG RETROSPOT,VINTAGE SNAKES & LADDERS
3 TIER CAKE TIN RED AND CREAM,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,TOY TIDY PINK POLKADOT,RECYCLING BAG RETROSPOT,VINTAGE SNAKES & LADDERS
5 HOOK HANGER MAGIC TOADSTOOL,FIVE HEART HANGING DECORATION,FRIDGE MAGNETS LES ENFANTS ASSORTED,5 HOOK HANGER MAGIC TOADSTOOL,FRIDGE MAGNETS US DINER ASSORTED,GINGHAM HEART DOORSTOP RED
60 TEATIME FAIRY CAKE CASES,60 TEATIME FAIRY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,HAND WARMER BIRD DESIGN,RED 3 PIECE RETROSPOT CUTLERY SET,BLUE 3 PIECE POLKADOT CUTLERY SET
"AIRLINE LOUNGE,METAL SIGN",PINK BREAKFAST CUP AND SAUCER,CERAMIC CHERRY CAKE MONEY BANK,CERAMIC STRAWBERRY CAKE MONEY BANK,TEA TIME DES TEA COSY,GIRLY PINK TOOL SET
ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE RED,INFLATABLE POLITICAL GLOBE,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES,STARS GIFT TAPE
ALARM CLOCK BAKELIKE IVORY,HAND WARMER BIRD DESIGN,BLACK HEART CARD HOLDER,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,60 TEATIME FAIRY CAKE CASES


In [99]:
neighbours.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries,  SET 2 TEA TOWELS I LOVE LONDON  to ZINC WILLIE WINKIE  CANDLE STICK
Data columns (total 5 columns):
1    212 non-null object
2    212 non-null object
3    212 non-null object
4    212 non-null object
5    212 non-null object
dtypes: object(5)
memory usage: 19.9+ KB
